<a href="https://colab.research.google.com/github/Lerity12/my-project/blob/main/A_B_Test_Significance_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Python_colab

/content/drive/MyDrive/Python_colab


# Підготовка даних

- Завантаження датасету з даними
- Виправлення формату даних, перевірка пропусків, перегляд форми таблиці

In [3]:
# завантаження таблиці

df = pd.read_csv("DataSet.csv")
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,session with orders,1
1,2020-11-03,New Zealand,desktop,Oceania,Direct,2,2,session with orders,1
2,2020-11-04,Bulgaria,mobile,Europe,Paid Search,2,1,session with orders,1
3,2020-11-04,Kuwait,mobile,Asia,Organic Search,2,2,session with orders,1
4,2020-11-05,Serbia,desktop,Europe,Social Search,2,2,session with orders,1


In [4]:
# детальна інформація датасету

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


In [5]:
# виправлення формату дати

df["date"] = pd.to_datetime(df["date"])

In [6]:
# перервірка формату

print(df.dtypes)

date          datetime64[ns]
country               object
device                object
continent             object
channel               object
test                   int64
test_group             int64
event_name            object
value                  int64
dtype: object


In [7]:
# перевірка на пропущені значення

df.isna().sum()

,0
date,0
country,0
device,0
continent,0
channel,0
test,0
test_group,0
event_name,0
value,0


In [8]:
# інформація по к-сті рядків і стовпців

print(df.shape)

(800996, 9)


In [9]:
# статистика

print(df.describe())

                                date           test     test_group  \
count                         800996  800996.000000  800996.000000   
mean   2020-12-07 00:33:34.715679488       2.831191       1.499608   
min              2020-11-01 00:00:00       1.000000       1.000000   
25%              2020-11-19 00:00:00       2.000000       1.000000   
50%              2020-12-05 00:00:00       3.000000       1.000000   
75%              2020-12-20 00:00:00       4.000000       2.000000   
max              2021-01-27 00:00:00       4.000000       2.000000   
std                              NaN       1.116940       0.500000   

               value  
count  800996.000000  
mean        9.475863  
min         1.000000  
25%         1.000000  
50%         2.000000  
75%         5.000000  
max      1575.000000  
std        37.493267  


# Автоматизація підрахунку статистичної значущості в A/B-тестах

## Мета:
Замість онлайн-калькуляторів реалізувати автоматичний підрахунок статистичної значущості для ключових конверсійних метрик A/B-тестів. Базовий розрахунок робиться в тоталі по кожному тесту (A vs B), а за потреби той самий код легко запускається в розрізах категорій.




In [10]:
# словник метрик
metrics = {
    "add_payment_info / session": ("add_payment_info", "session"),
    "add_shipping_info / session": ("add_shipping_info", "session"),
    "begin_checkout / session":    ("begin_checkout",  "session"),
    "new_accounts / session":      ("new account",     "session"),
}

# рівень значущості
alpha = 0.05


# підготовка зведеної таблиці з лічильниками подій по тестах і групах
pivot_df = (
    df.pivot_table(
        index=["test", "test_group"],
        columns="event_name",
        values="value",
        aggfunc="sum",
        fill_value=0
    )
    .reset_index()
)

results_rows = []


# прохід циклом по кожному тесту
for test_number, test_slice in pivot_df.groupby("test"):
    test_slice = test_slice.set_index("test_group")
    if not {1, 2}.issubset(test_slice.index):
        continue

     # для кожної метрики рахуємо конверсії та p-value
    for metric_name, (numerator_event, denominator_event) in metrics.items():

      # control (group 1)
      numerator_count_control = int(test_slice.at[1, numerator_event]) if numerator_event in test_slice.columns else 0
      denominator_count_control = int(test_slice.at[1, denominator_event]) if denominator_event in test_slice.columns else 0


      # variant (group 2)
      numerator_count = int(test_slice.at[2, numerator_event]) if numerator_event in test_slice.columns else 0
      denominator_count = int(test_slice.at[2, denominator_event]) if denominator_event in test_slice.columns else 0

      # конверсії у вигляді часток
      conversion_rate_control = (
            numerator_count_control / denominator_count_control
            if denominator_count_control else np.nan
        )
      conversion_rate = (
            numerator_count / denominator_count
            if denominator_count else np.nan
        )


      #відносна зміна
      metric_change = (
            (conversion_rate - conversion_rate_control) / conversion_rate_control * 100
            if (pd.notna(conversion_rate_control) and conversion_rate_control)
            else np.nan
        )


      #двовибірковий Z-тест для пропорцій (перевіряємо, чи різниця не випадкова)
      if denominator_count_control > 0 and denominator_count > 0:
            z_stat, p_value = proportions_ztest(
                count=[numerator_count, numerator_count_control],
                nobs=[denominator_count, denominator_count_control],
                alternative="two-sided"
            )
      else:
          z_stat, p_value = np.nan, np.nan



      # записуємо рядок з результатами
      results_rows.append({
          "test_number": test_number,
          "metric": metric_name,
          "numerator_event": numerator_event,
          "denominator_event": denominator_event,

          "numerator_count_control": numerator_count_control,
          "denominator_count_control": denominator_count_control,
          "conversion_rate_control":   conversion_rate_control,

          "numerator_count": numerator_count,
          "denominator_count": denominator_count,
          "conversion_rate":   conversion_rate,

          "metric_change": metric_change,
          "z_stat": z_stat,
          "p_value": p_value,
          "significant": (p_value < alpha) if pd.notna(p_value) else False
          })


# перетворюємо список рядків у DataFrame
result_df = pd.DataFrame(results_rows)



result_df["conversion_rate_control"] = result_df["conversion_rate_control"].round(6)
result_df["conversion_rate"] = result_df["conversion_rate"].round(6)
result_df["metric_change"] = result_df["metric_change"].round(6)
result_df["z_stat"] = result_df["z_stat"].round(6)
result_df["p_value"] = result_df["p_value"].round(8)



result_df.head(10)

,test_number,metric,numerator_event,denominator_event,numerator_count_control,denominator_count_control,conversion_rate_control,numerator_count,denominator_count,conversion_rate,metric_change,z_stat,p_value,significant
0,1,add_payment_info / session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,3.924884,0.000087,True
1,1,add_shipping_info / session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,2.603571,0.009226,True
2,1,begin_checkout / session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,2.978783,0.002894,True
3,1,new_accounts / session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,-1.542883,0.122859,False
4,2,add_payment_info / session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,1.240994,0.214608,False
5,2,add_shipping_info / session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,0.709557,0.477979,False
6,2,begin_checkout / session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,0.952898,0.340642,False
7,2,new_accounts / session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,0.588793,0.556000,False
8,3,add_payment_info / session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,0.643172,0.520112,False
9,3,add_shipping_info / session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,-1.413727,0.157442,False


In [11]:
# експорт файлу
result_df.to_csv("/content/drive/MyDrive/ab_test_results.csv", index=False)

## Опис результату:

- На цьому етапі реалізовано Python-скрипт для автоматичного розрахунку статистичної значущості результатів A/B-тестів.


- Код формує зведену таблицю з подіями для кожного тесту та групи (контрольної й варіантної), обчислює конверсії, відносну зміну метрики та p-value за допомогою двовибіркового Z-тесту для пропорцій.


- Результатом є таблиця з основними показниками кожного тесту (конверсія, зміна метрики, статистика Z, p-value, значущість), яку можна використовувати для подальшого аналізу в різних розрізах за тестом, країною, пристроєм



# Висновки:


- Дві з чотирьох протестованих метрик показали статистично значущі результати.

- Конверсія зросла для метрик “begin_checkout / session” та “add_payment_info / session”.

- Отримані результати свідчать, що покращення процесу оформлення замовлення та введення платіжної інформації найбільше вплинули на поведінку користувачів.

**Наступні кроки:**

- Перевірити результати на більшій вибірці даних.

- Реалізувати автоматизацію відстеження A/B-тестів у Tableau.



# Посилання на файли проекту

[Tableau Dashboard](https://public.tableau.com/views/Book2_17597679341350/ABTestSignificanceOverview?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

[CSV results file](https://drive.google.com/file/d/1x3FsjkmU_O7y59h1Po9YsbRpZR5JRpPh/view?usp=sharing)
